In [28]:
from rich import print
from typing import Any

## Основы

### Синтаксис

In [ ]:
class MyClass:  # Определение класса
    """docstring"""        # Документация (__doc__)

    # Только ДЛЯ СВОЙСТВ ЭКЗЕМПЛЯРА
    __slots__ = ('a', '_b', '__c')

    x = 0  # Свойство класса (__dict__)

    def __init__(          # Double UNDERscore methods
        self,
        a: str,
        b: str,
        c: str,
    ): # Свойства объекта
        self.a: str = a    # Public

        # Инкапсуляция
        self._b: str = b   # Protected (Условно)
        self.__c: str = c  # Private   (_MyClass__c)

    def foo(self):    # Методы класса
        print(self)   # Ссылка на экземпляр класса

    @classmethod      # Работает только со свойствами Класса
    def cls_method(cls):
        print(cls)

    @staticmethod     # Не имеет доступа к свойствам Класса/Экземпляра
    def static_method():
        print(MyClass)

    @property
    def c(self) -> str:
        return self.__c

    @c.setter
    def c(self, value: str):
        self.__c = value

    print('Выполнится до создания объекта')

Выполнится до создания объекта

### Словарь атрибутов класса

In [21]:
print(MyClass.__dict__)

mappingproxy({
    '__module__': '__main__',
    '__doc__': 'docstring',
    'x': 0,
    '__init__': <function MyClass.__init__ at 0x10b3e27a0>,
    'foo': <function MyClass.foo at 0x10b3e2840>,
    'c': <property object at 0x10b3dc900>,
    '__dict__': <attribute '__dict__' of 'MyClass' objects>,
    '__weakref__': <attribute '__weakref__' of 'MyClass' objects>
})

### CRUD свойств класса

In [22]:
setattr(MyClass, 'prop', 'tmp_prop')  # MyClass.newProp = 1

print(
    hasattr(MyClass, 'prop'),         # Проверка существования атрибута
    getattr(MyClass, 'prop', None),   # MyClass.prop3 -> Error if None
    sep='\n',
)

delattr(MyClass, 'prop')              # del MyClass.prop3 -> Error if None

True
tmp_prop

### Экземпляры класса

In [ ]:
obj = MyClass(a='a', b='b', c='c')  # Экземпляр класса
print(
    type(obj),                 # Тип объекта (Класс)
    type(obj) is MyClass,      # Является ли типом MyClass?
    isinstance(obj, MyClass),  # Наследуется ли от MyClass? 
    sep='\n',
)

<class '__main__.MyClass'>
True
True

### accessify
Позволяет создать полностью защищенный метод класса

In [ ]:
from accessify import private, protected


class MyClass:
    @private    # Только внутри Класса
    def private_method(self):
        print('This is a private method')

    @protected  # Класс/Дочерние классы
    def protected_method(self):
        print('This is a protected method')

    def public_method(self):
        print('This is a public method')
        self.private_method()
        self.protected_method()


# Использование
obj = MyClass()
obj.public_method()       # Работает нормально

# obj.private_method()    # Вызовет ошибку, если попытаться вызвать напрямую
# obj.protected_method()  # Также вызовет ошибку при прямом вызове

This is a public method

This is a private method

This is a protected method

### Декораторы классов

In [66]:
from typing import TypeVar

T = TypeVar('T')


# Функция-декоратор
def decoFn(C: T) -> T:
    print('Декоратор decoFn сработал')
    return C


# Класс-декоратор
class DecClass:
    def __init__(self, c: Any):
        print('Декоратор DecClass сработал')
        self._c = c

    def __call__(self, *args: tuple[Any], **kwargs: dict[str, Any]):
        print('__call__ called')
        self._c(*args, **kwargs)

In [67]:
@decoFn
class Class0: pass   # noqa

@DecClass
class  Class1: pass  # noqa

tmp_obj_0 = Class0()
tmp_obj_1 = Class1()

Декоратор decoFn сработал

Декоратор DecClass сработал

__call__ called

## Обработка доступа к свойствам экземпляра

In [ ]:
class CustomClass:
    
    # При присвоении значения атрибуту
    def __setattr__(self, key: str, value: str):
        print(f'__setattr__: {key} = {value}')
        # self.name = value ВЫЗЫВАЕТ РЕКУРСИВНОЕ ПРИСВАИВАНИЕ
        return object.__setattr__(self, key, value)
    
    # При обращении к атрибуту
    # Если => AttributeError, то __getattr__()
    def __getattribute__(self, item: str):
        print(f'__getattribute__: {item}')
        return object.__getattribute__(self, item)
    
    # При обращении к несуществующему атрибуту
    def __getattr__(self, item: str):
        print(f'__getattr__: {item} NOT FOUND')
        return None
    
    # При попытке удаления атрибута
    def __delattr__(self, item: str):
        print(f'__delattr__: {item} removed')
        object.__delattr__(self, item)

In [54]:
custom_object = CustomClass()
custom_object.a
custom_object.a = 'property'
custom_object.a
del custom_object.a

__getattribute__: a

__getattr__: a NOT FOUND

__setattr__: a = property

__getattribute__: a

__delattr__: a removed

## Наследование

### Прямое

In [ ]:
class Base:  # Базовый класс
    def __init__(self):
        print('Base.__init__()')

    def foo(self):
        print('Base.foo()')


class Child(Base):  # Класс Child наследуется от Base
    def __init__(self):
        print('Child.__init__()')

        # Вызов конструктора Base
        super().__init__()  # Base.__init__(self)
                            # super(Child, self).__init__()

    def func1(self):
        print('Child.foo()')
        Child.func1(self)


child_obj = Child()
child_obj.foo()

Child.__init__()

Base.__init__()

Base.foo()

### Множественное
Поиск слева направо. Останавливается на первом найденном.

In [74]:
class Base:          # Базовый класс
    def func1(self):
        print('Метод func1() класса Base')


class Child0(Base):  # Класс Child0 наследует класс Base
    def func2(self):
        print('Метод func2() класса Child0')


class Child1(Base):  # Класс Child1 наследует класс Base
    def func1(self):
        print('Метод func1() класса Child1')

    def func2(self):
        print('Метод func2() класса Child1')

    def func3(self):
        print('Метод func3() класса Child1')

    def func4(self):
        print('Метод func4() класса Child1')


class MultiChild(Child0, Child1):  # Множественное наследование
    def func4(self):
        print('Метод func4() класса MultiChild')


c = MultiChild()
c.func1()  # Child1
c.func2()  # Child0
c.func3()  # Child1
c.func4()  # MultiChild

# __bases__ используется для получения базовых классов
print(Base.__bases__)
print(Child0.__bases__)
print(Child1.__bases__)
print(MultiChild.__bases__)

Метод func1() класса Child1

Метод func2() класса Child0

Метод func3() класса Child1

Метод func4() класса MultiChild

(<class 'object'>,)

(<class '__main__.Base'>,)

(<class '__main__.Base'>,)

(<class '__main__.Child0'>, <class '__main__.Child1'>)

### MRO

In [ ]:
class Class1: pass
class Class2(Class1): pass
class Class3(Class2): pass
class Class4(Class3): pass
class Class5(Class2): pass
class Class6(Class5): pass
class Class7(Class4, Class6):pass
c = Class7()

#__mro__ - цепочка наследования
print(Class7.__mro__)

(
    <class '__main__.Class7'>,
    <class '__main__.Class4'>,
    <class '__main__.Class3'>,
    <class '__main__.Class6'>,
    <class '__main__.Class5'>,
    <class '__main__.Class2'>,
    <class '__main__.Class1'>,
    <class 'object'>
)

## Абстракция

### Без декоратора

In [86]:
class AbstractClass:
    def abstract_method(self) -> str:  # noqa
        raise NotImplementedError('Надо бы переопределить')

class SomeClass(AbstractClass):
    def abstract_method(self) -> str:
        return 'string'
    
c = SomeClass()
c.abstract_method()

'string'

### С декоратором

In [96]:
from abc import ABCMeta, abstractmethod

class ABClass(metaclass=ABCMeta):
    @abstractmethod  # Абстрактный метод
    def foo(self):
        pass

    @classmethod     # Абстрактный метод КЛАССА
    @abstractmethod
    def class_foo(cls):
        print('Abstract static method')
    
    @staticmethod   # Абстрактный статический метод
    @abstractmethod
    def static_foo():
        print('Abstract static method')

class Child(ABClass):
    def foo(self): print('fooooo')

    @classmethod
    def class_foo(cls): pass

    @staticmethod
    def static_foo(): pass

try:
    c = Child()
    c.foo()
except TypeError:
    print('Not implemented methods')

fooooo

## DUnder methods

[Документация](https://docs.python.org/3/reference/datamodel.html#special-method-names)

### `__new__`/`__init__`/`__del__`
Выполняется до инициализации объекта

In [116]:
class SomeClass:
    # До создания объекта
    def __new__(cls):  # Должен возвращать адрес объекта
        print('__new__ called')
        print(super)
        
        # object.__new__(cls) создает экземпляр класса
        return super().__new__(cls)  # => new object address

    # После создания объекта
    def __init__(self):
        self.x = 10
        print('__init__ called')

    # Перед удалением
    def __del__(self):
        print("__del__ called")

In [117]:
obj = SomeClass()

del obj

__new__ called

<class 'super'>

__init__ called

__del__ called

### Прочие

```python
__len__(self)        # при использовании функции len()
__bool__(self)       # при использовании функции bool();
__int__(self)        # при преобразовании с помощью функции int();
__float__(self)      # при преобразовании с помощью функции float();
__complex__(self)    # при преобразовании с помощью функции complex();
__round__(self, n)   # при использовании функции round();
__index__(self)      # при использовании функций bin(), hex() и oct();
__repr__(self)       # при выводе в интерактивной оболочке или repr()
__str__(self)        # при print() или str()
__hash__(self)       # Если экземпляр используется как значение dict||tuple
```

### Сравнение

```python
x == y    # равно: x.__eq__(y);
x != y    # не равно: x.__ne__(y);
x < y     # меньше: x.__lt__(y);
x > y     # больше: x.__gt__(y);
x <= y    # меньше или равно: x.__le__(y);
x >= y    # больше или равно: x.__ge__(y);
y in x    # проверка на вхождение: x.__contains__(y).
```

### Математические

```python
x + y    # x.__add__(y)
y + x    # x.__radd__(y)  Экземпляр справа
x += y   # x.__iadd__(y)
x — y    # x.__sub__(y)
y — x    # x.__rsub__(y)  Экземпляр справа
x -= y   # x.__isub__(y)
x * y    # x.__mul__(y)
y * x    # x.__rmul__(y)  Экземпляр справа
x *= y   # x.__imul__(y)
x / y    # x.__truediv__(y)
y / x    # x.__rtruediv__(y)   Экземпляр справа
x /= y   # x.__itruediv__(y)
x // y   # x.__floordiv__(y)
y // x   # x.__rfloordiv__(y)  Экземпляр справа
x //= y  # x.__ifloordiv__(y)
x % y    # x.__mod__(y)
y % x    # x.__rmod__(y)  Экземпляр справа
x %= y   # x.__imod__(y)
x  y   # x.__pow__(y)
y  x   # x.__rpow__(y)  Экземпляр справа
x = y  # x.__ipow__(y)
-x       # x.__neg__()
+x       # x.__pos__()
abs(x)   # x.__abs__()
```

### Двоичные

```python
~x       # двоичная инверсия: x.__invert__();
x & y    # двоичное И: x.__and__(y);
y & x    # двоичное И (экземпляр класса справа): x.__rand__(y);
x &= y   # двоичное И и присваивание: x.__iand__(y);
x | y    # двоичное ИЛИ: x.__or__(y);
y | x    # двоичное ИЛИ (экземпляр класса справа): x.__ror__(y);
x |= y   # двоичное ИЛИ и присваивание: x.__ior__(y);
x ^ y    # двоичное исключающее ИЛИ: x.__xor__(y);
y ^ x    # двоичное исключающее ИЛИ (экземпляр класса справа): x.__rxor__(y);
x ^= y   # двоичное исключающее ИЛИ и присваивание: x.__ixor__(y);
x << y   # сдвиг влево: x.__lshift__(y);
y << x   # сдвиг влево (экземпляр класса справа): x.__rlshift__(y);
x <<= y  # сдвиг влево и присваивание: x.__ilshift__(y);
x >> y   # сдвиг вправо: x.__rshift__(y);
y >> x   # сдвиг вправо (экземпляр класса справа): x.__rrshift__(y);
x >>= y  # сдвиг вправо и присваивание: x.__irshift__(y).
```

## Дексрипторы

```python
# Встроенные дескрипторы
property            # DD для управляемых атрибутов
staticmethod        # NDD для статических методов
classmethod         # NDD для методов класса
function            # NDD (обычные методы становятся bound methods)
super               # NDD для доступа к родительскому классу
cached_property     # NDD с кэшированием
member_descriptor   # DD для __slots__
getset_descriptor   # Внутренний дескриптор для C-расширений
```

❗ Порядок разрешения атрибутов:\
1. `__getattribute__` класса объекта\
2. `property`/`Data Descriptor`\
3. `__dict__`\
4. `Non Data Descriptor`\
5. `Class Attr`\
4. `__getattr__`

### Протокол дескрипторов
- `__get__(self, instance, owner)`: Управляет доступом к атрибуту. Возвращает значение атрибута.
  - `instance`: экземпляр класса, для которого был запрошен атрибут.
  - `owner`: класс экземпляра.
- `__set__(self, instance, value)`: Устанавливает значение атрибута.
  - `instance`: экземпляр класса, атрибут которого устанавливается.
  - `value`: значение, присваиваемое атрибуту.
- `__delete__(self, instance)`: Удаляет атрибут.
  - `instance`: экземпляр класса, атрибут которого удаляется.

### Виды дескрипторов
- Non-data дескрипторы: Имеют только метод `__get__`. Могут быть переопределены атрибутами экземпляра.
- Data дескрипторы: Имеют методы `__set__` и/или `__delete__`. Имеют приоритет над атрибутами экземпляра.